# **Google Vision Product Search:** 
## Build a Product catalog recognition engine in one hour

Vision API Product Search allows retailers to create products, each containing reference images that visually describe the product from a set of viewpoints. Retailers can then add these products to product sets. Currently Vision API Product Search supports the following product categories: homegoods, apparel, toys, packaged goods, and general .

When users query the product set with their own images, Vision API Product Search applies machine learning to compare the product in the user's query image with the images in the retailer's product set, and then returns a ranked list of visually and semantically similar results.

After loading your catalog into Vision Product Search, you'll be able to search for similar products in your catalog by providing a image

> This notebook is using a Kaggle dataset for product recognition. The goal is to extract a csv for bulk import in Product Vision Search  

**Useful links:**  
https://github.com/zinjiggle/google-product-search-simple-ui  
https://github.com/GoogleCloudPlatform/python-docs-samples/tree/master/vision/cloud-client/product_search
https://github.com/googleapis/python-vision/tree/main/samples

## 0. Install vision library

In [2]:
pip install google-cloud-vision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade google-cloud-storage

  Using cached google_cloud_storage-2.5.0-py2.py3-none-any.whl (106 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.6.0
    Uninstalling google-cloud-core-1.6.0:
      Successfully uninstalled google-cloud-core-1.6.0
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.37.1
    Uninstalling google-cloud-storage-1.37.1:
      Successfully uninstalled google-cloud-storage-1.37.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigtable 1.7.0 requires google-cloud-core<2.0dev,>=1.4.1, but you have google-cloud-core 2.3.2 which is incompatible.
-oogle-cloud-datastore 2.1.0 requires google-cloud-core<2.0dev,>=1.4.0, but you have google-cloud-core 2.3.2 which is incompatible.
jupyterlab-bigq

In [1]:
project_id='pod-fr-retail'
location='europe-west1'
product_set='kaggle_shoes'
bucket_name="pod-fr-retail-kaggle"
gcs_bucket="gs://"+bucket_name+"/"

## 1. Create a Products catalog with Bigquery
The first step shows how to prepare a product catalog for **Vision Product Search** in BigQuery:
* **image-uri**: The Google Cloud Storage URI of the reference image.
* **image-id**: Optional. A unique value if you supply it. Otherwise, the system will assign a unique value.
* **product-set-id**: A unique identifier for the product set to import the images into.
* **product-id**: A user-defined ID for the product identified by the reference image. A product-id can be associated with multiple reference images. Note: A single product may also belong to several product sets. If a product-id already exists on bulk import then product-category, product-display, and labels are ignored for that line entry.
* **product-category**: Allowed values are homegoods-v2, apparel-v2, toys-v2, packagedgoods-v1, and general-v1 *; the category for the product identified by the reference image. Inferred by the system if not specified in the create request. Allowed values are also listed in the productCategory reference documentation. Legacy productCategory codes: Legacy categories (homegoods, apparel, and toys) are still supported, but the updated -v2 categories should be used for new products.
* **product-display-name**: Optional. If you don't provide a name for the product displayName will be set to " ". You can update this value later.
* **labels**: Optional. A string (with quotation marks) of key-value pairs that describe the products in the reference image. For example:"color=black,style=formal"
* **bounding-poly**: Optional. Specifies the area of interest in the reference image. If a bounding box is not specified: Bounding boxes for the image are inferred by the Vision API; multiple regions in a single image may be indexed if multiple products are detected by the API. The line must end with a comma. See the example below for a product without a bounding poly specified.If you include a bounding box, the boundingPoly column should contain an even number of comma-separated numbers, with the format p1_x,p1_y,p2_x,p2_y,...,pn_x,pn_y. An example line looks like this: 0.1,0.1,0.9,0.1,0.9,0.9,0.1,0.9.

#### 1.1 Product Catalog exploration (Kaggle dataset) 

Let's start by exploring our data. We are using a product images catalog from a kaggle contest. Goal was to classify automatically product based on a image. The dataset contains 48 products main categories and around 12 millions images. For our notebook quickstart, we'll focus on Shoes category.

In [2]:
%%bigquery
SELECT category_level1, count(*) num_products  
FROM 
    `pod-fr-retail.kaggle.train_images` a 
JOIN `pod-fr-retail.kaggle.category_names`  b 
ON CAST(b.category_id AS STRING) =(REGEXP_EXTRACT(a.path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([^-]*)'))
GROUP BY 1 ORDER BY 2 desc
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.93rows/s]


,category_level1,num_products
0,TELEPHONIE - GPS,1227001
1,AUTO - MOTO,1193619
2,INFORMATIQUE,1124907
3,DECO - LINGE - LUMINAIRE,1111509
4,LIBRAIRIE,863965
5,BIJOUX - LUNETTES - MONTRES,688243
6,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,620366
7,JEUX - JOUETS,551408
8,SPORT,434791
9,BAGAGERIE,434675


In shoes category

In [3]:
%%bigquery
SELECT category_level2
from `pod-fr-retail.kaggle.category_names` 
WHERE category_level1 like 'CHAUSSURES%'
GROUP BY 1

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.15rows/s]


,category_level2
0,BOTTES - BOTTINES
1,CHAUSSURES DETENTE
2,BASKET - SPORTSWEAR
3,CHAUSSURES DE VILLE
4,ACCESSOIRES CHAUSSURES


#### 1.2 Create a table with the appropriate schema from Product Catalog (Kaggle dataset) 

In [4]:
%%bigquery
CREATE OR REPLACE TABLE `pod-fr-retail.kaggle.products_vision_search` AS
SELECT 
    a.* EXCEPT (category_id)
    ,CONCAT(replace(lower(CONCAT('','cl1=',b.category_level1,',cl2=',b.category_level2,',cl3=',b.category_level3,' ')),' ','')) labels
    ,null as poly
FROM (
    SELECT 
        path_to_images image_uri
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([0-9]*-[0-9]*-[0-9]*)')) AS image_id
        ,'kaggle_shoes' as product_set_id
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/[0-9]*-([0-9]*)')) AS product_id
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/([^-]*)')) AS category_id
        ,'apparel-v2' product_category
        ,(REGEXP_EXTRACT(path_to_images,r'gs://pod-fr-retail-kaggle/train-images/[0-9]*/[0-9]*-([0-9]*)')) AS product_display_name
    FROM `pod-fr-retail.kaggle.train_images`
  ) a 
JOIN (SELECT * FROM `pod-fr-retail.kaggle.category_names` 
      WHERE 
      #category_level1='HYGIENE - BEAUTE - PARFUM'
      #category_level1='CHAUSSURES - ACCESSOIRES'
      category_level1 like 'CHAUSSURES%'
  ) b 
ON CAST(b.category_id AS STRING) =a.category_id
#WHERE rand()<0.10;ß

Query complete after 0.00s: 100%|██████████| 5/5 [00:00<00:00, 2027.80query/s]                        


""


In [5]:
%%bigquery
select count(*) number_of_products from `pod-fr-retail.kaggle.products_vision_search`

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.26s/rows]


,number_of_products
0,369325


# 2. Prepare bulk files to create a new set Data extraction (export CSV for bulk import)
## Extract from BigQuery to Google Cloud Storage and split file in multiple files (20k lines max per file)

In [6]:
%%bigquery
SELECT * FROM `pod-fr-retail.kaggle.products_vision_search` LIMIT 2

Downloading: 100%|██████████| 2/2 [00:01<00:00,  1.58rows/s]


,image_uri,image_id,product_set_id,product_id,product_category,product_display_name,labels,poly
0,gs://pod-fr-retail-kaggle/train-images/1800000...,1000010547-6222070-0,kaggle_shoes,6222070,apparel-v2,6222070,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN
1,gs://pod-fr-retail-kaggle/train-images/5700000...,1000010547-18838943-3,kaggle_shoes,18838943,apparel-v2,18838943,"cl1=chaussures-accessoires,cl2=bottes-bottines...",NaN


In [1]:
!gcloud auth -q configure-docker europe-west1-docker.pkg.dev


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "europe-west1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: europe-west1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [8]:
import docker
client = docker.from_env()

In [10]:
client.images.pull("europe-west1-docker.pkg.dev/pod-fr-retail/demok8/demo_k8_jobs_bqs_split", "latest")

<Image: 'europe-west1-docker.pkg.dev/pod-fr-retail/demok8/demo_k8_jobs_bqs_split:latest'>

In [11]:
image=client.images.get('europe-west1-docker.pkg.dev/pod-fr-retail/demok8/demo_k8_jobs_bqs_split:latest')
image.attrs["VirtualSize"]

167999550

In [12]:
container=client.containers.run("europe-west1-docker.pkg.dev/pod-fr-retail/demok8/demo_k8_jobs_bqs_split:latest"
                                ,["python3",  "-u", "/bq_split.py"
                                    ,"--projectId", "pod-fr-retail"
                                    ,"--dataset", "kaggle"
                                    ,"--tableId", "products_vision_search"
                                    ,"--selected_fields", "image_uri,image_id,product_set_id,product_id,product_category,product_display_name,labels,poly"
                                    ,"--bucket_name", "pod-fr-retail-kaggle"
                                    ,"--file_prefix", "products_vision_search/kaggle_shoes/extract-"
                                    ,"--max_rows", "20000"
                                  ]
                                , detach=True
                                , stdout=True
                                , stream=True)
for log in container.logs(stream=True, stdout=True, stderr=True):
    print(log.decode("utf-8"))

================ Start ==================



Start extract into pod-fr-retail-kaggle.products_vision_search/kaggle_shoes/extract- folder 



pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-0 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-1 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-2 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-3 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-4 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-5 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-6 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-7 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-8 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-9 created

pod-fr-retail-kaggle/products_vision_search/kaggle_shoes/extract-10 created

pod-fr-retail-kaggle/p

# 3. Product set creation
Bulk import every files to Vision Product Search (to a specific product set id specified in the file itself)

## 3.1 List all files

In [16]:
from google.cloud import storage
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blobs = client.list_blobs(bucket_name, prefix=file_prefix)
gcs_uris=[]
for item in blobs:
    gcs_uris.append(item.name)
    print(item.name)


products_vision_search/kaggle_shoes/extract-0
products_vision_search/kaggle_shoes/extract-1
products_vision_search/kaggle_shoes/extract-10
products_vision_search/kaggle_shoes/extract-11
products_vision_search/kaggle_shoes/extract-12
products_vision_search/kaggle_shoes/extract-13
products_vision_search/kaggle_shoes/extract-14
products_vision_search/kaggle_shoes/extract-15
products_vision_search/kaggle_shoes/extract-16
products_vision_search/kaggle_shoes/extract-17
products_vision_search/kaggle_shoes/extract-18
products_vision_search/kaggle_shoes/extract-2
products_vision_search/kaggle_shoes/extract-3
products_vision_search/kaggle_shoes/extract-4
products_vision_search/kaggle_shoes/extract-5
products_vision_search/kaggle_shoes/extract-6
products_vision_search/kaggle_shoes/extract-7
products_vision_search/kaggle_shoes/extract-8
products_vision_search/kaggle_shoes/extract-9


## 3.2 Import files and store responses (operation) 


In [17]:
from google.cloud import vision_v1
responses=[]
client = vision_v1.ProductSearchClient()
parent = client.location_path(project_id, location)

def callback(operation_future):
        # Handle result.
        result = operation_future.result()
        
for gcs_uri in gcs_uris:
    gcs_source = vision_v1.types.ImportProductSetsGcsSource(csv_file_uri = gcs_bucket+gcs_uri)
    input_config = vision_v1.types.ImportProductSetsInputConfig(gcs_source=gcs_source)
    response = client.import_product_sets(parent, input_config)
    response.add_done_callback(callback)
    responses.append(response)

In [18]:
for response in responses:
    print(response.operation.name, ':')
    print(response.metadata)
    print('--------------------')

projects/pod-fr-retail/locations/europe-west1/operations/8a6dfdb41d59ecf2 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nanos: 424273764
}

--------------------
projects/pod-fr-retail/locations/europe-west1/operations/612dfcb9b1dc60e5 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nanos: 485121446
}

--------------------
projects/pod-fr-retail/locations/europe-west1/operations/604d198c5a4cb290 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nanos: 549951186
}

--------------------
projects/pod-fr-retail/locations/europe-west1/operations/fc5bfd9763410407 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nanos: 608693558
}

--------------------
projects/pod-fr-retail/locations/europe-west1/operations/d15cbf3020db4703 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nanos: 651175341
}

--------------------
projects/pod-fr-retail/locations/europe-west1/operations/c70c423510dde541 :
state: PROCESSING
submit_time {
  seconds: 1666604631
  nan

## 3.3 Check index time of product sets 

In [21]:
import time
from google.cloud import vision_v1
client = vision_v1.ProductSearchClient()
parent = client.location_path(project_id, location)
name = client.product_set_path(project_id, location, product_set)
for element in client.list_product_sets(parent):
    print(element)
    print('Last index:',time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(element.index_time.seconds)))
    print('------')
    pass
#print(client.list_products_in_product_set(name))

name: "projects/pod-fr-retail/locations/europe-west1/productSets/kaggle_shoes"
display_name: " "
index_time {
  seconds: 1666599567
  nanos: 889620900
}
index_error {
}

Last index: 2022-10-24 08:19:27
------
name: "projects/pod-fr-retail/locations/europe-west1/productSets/pvs_shoes"
display_name: " "
index_time {
  seconds: 1666599567
  nanos: 889620900
}
index_error {
}

Last index: 2022-10-24 08:19:27
------
